In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import torch
torch.cuda.empty_cache()

In [2]:
import torch
from torch import optim
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10, Food101
from torchvision.transforms import v2
from pathlib import Path
import wandb
import os
import sys

sys.path.append("../src")

from trainer import Trainer
from module import ResNetModule
from utils import model_size, load_from_checkpoint
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset

In [3]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [4]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "ResNet9",
        "num_model_layers": 9,
        "batch_size": 128,
        "max_epochs": 20,
        "dataset": "Food101",
        "optimizer": {
            "name": "Adam",
            "weight_decay": 1e-4
        },
        "lr_scheduler": {
            "max_lr": 0.01
        },
        "train_split": 0.7,
        "val_split": 0.3
    },
    logs_path=logs_path,
    offline=False,
    notes="Training on Food101 dataset"
)

In [5]:
# cpu_count = os.cpu_count()
cpu_count = 7
dataset = Food101(data_path, split="train", download=True)

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]]
)

normalize_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465),
                 std=(0.2023, 0.1994, 0.2010))
])

val_transforms = v2.Compose([
    # Resize
    v2.Resize(255),
    v2.CenterCrop(224),

    normalize_transforms
])


train_dataset = MapDataset(train_dataset, transform=v2.Compose([
    # Data auguments
    v2.RandomResizedCrop(224),
    v2.RandomHorizontalFlip(),

    normalize_transforms
]))

val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True, num_workers=cpu_count, pin_memory=True)
val_dataloader = DataLoader(
    val_dataset, batch_size=logger.config["batch_size"],  num_workers=cpu_count, pin_memory=True)

In [6]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=80.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=1, batch_size=5, max_epochs=500, augument_data=False)
]

In [7]:
module = ResNetModule(toy_model=False)
optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=logger.config["optimizer"]["weight_decay"]
)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=logger.config["lr_scheduler"]["max_lr"],
    epochs=logger.config["max_epochs"],
    steps_per_epoch=len(train_dataloader)
)
# lr_scheduler = None

In [8]:
# module.model, optimizer = load_from_checkpoint(
#     path="../logs/wandb/offline-run-20241215_132918-77n093vj/checkpoints/best.pt",
#     model=module.model,
#     optimizer=optimizer
# )

summary(module.model, input_size=(train_dataset[0][0].shape), batch_size=logger.config["batch_size"], device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 224, 224]           1,728
       BatchNorm2d-2        [128, 64, 224, 224]             128
              ReLU-3        [128, 64, 224, 224]               0
         ConvBlock-4        [128, 64, 224, 224]               0
            Conv2d-5       [128, 128, 224, 224]          73,728
       BatchNorm2d-6       [128, 128, 224, 224]             256
              ReLU-7       [128, 128, 224, 224]               0
         MaxPool2d-8       [128, 128, 112, 112]               0
         ConvBlock-9       [128, 128, 112, 112]               0
           Conv2d-10       [128, 128, 112, 112]         147,456
      BatchNorm2d-11       [128, 128, 112, 112]             256
             ReLU-12       [128, 128, 112, 112]               0
           Conv2d-13       [128, 128, 112, 112]         147,456
      BatchNorm2d-14       [128, 128, 1

In [9]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True,
    lr_scheduler=lr_scheduler,
    lr_scheduler_on_epoch=False,
    checkpoint="best_val",
    num_workers=cpu_count
)

In [10]:
module.device

'cuda'

In [11]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Run stopped!


lr,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇█
step_train_accuracy,▂▃▆▂▇▅▃▃▂▁▃▃▁▁▂▂▂▃▁▁▂▆▅▃▂▅▂▁▅▆▅▆▂▂▂▂█▁▆▁
step_train_loss,█▆▄▂▃▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇██
lr,0.00053
model_architecture,ResNet9( (feature_...
step_train_accuracy,1.5625
step_train_loss,4.64147
training_step,187


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)